<a href="https://colab.research.google.com/github/yhc0712/which_day_to_buy_0050TW/blob/main/%E5%93%AA%E5%A4%A9%E5%AE%9A%E6%9C%9F%E5%AE%9A%E9%A1%8D%E7%94%B3%E8%B3%BC0050.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 定期定額要設定星期幾申購0050？

1. Weblink of data source
[Yahoo Finance](https://finance.yahoo.com/quote/0050.TW/)



In [ ]:
!pip install yfinance
import yfinance as yf
import h5py
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
tt50_info = yf.Ticker("0050.TW")
tt50_price = tt50_info.info['regularMarketPrice']

'''
從yahoo finance下載0050歷史資料
並新增工作日欄位
'''

tt50 = yf.download('0050.TW', start = '2002-06-25', end = datetime.today())

tt50['Weekday'] = [i.isoweekday() for i in tt50.index]

wd_convert = {
    1: '星期一',
    2: '星期二',
    3: '星期三',
    4: '星期四',
    5: '星期五'
}

[*********************100%***********************]  1 of 1 completed


isoweekday是以星期一為1，星期五為5，以此類推

先檢視歷史資料中交易之工作日，可以看到有包含星期六

In [ ]:
tt50['Weekday'].unique()

array([3, 4, 5, 1, 2, 6])

檢視星期六為交易日的日數，通常應該是補班日才會在星期六交易

故在本筆記本的分析內，將排除星期六申購的可能

In [ ]:
tt50[tt50['Weekday'] == 6]

,Open,High,Low,Close,Adj Close,Volume,Weekday
Date,,,,,,,
2016-01-30,59.590000,59.590000,59.590000,59.590000,59.590000,0,6
2016-06-04,63.980000,63.980000,63.980000,63.980000,63.980000,0,6
2016-09-10,69.790001,69.790001,69.790001,69.790001,69.790001,0,6
2017-02-18,73.480003,73.480003,73.480003,73.480003,73.480003,0,6
2017-06-03,77.330002,77.330002,77.330002,77.330002,77.330002,0,6


## 定義投資物件

In [ ]:
class Investment():
  def __init__(self, weekday:int, payment:int, start_date, quote):
    self.weekday: int = weekday
    self.payment = payment
    self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
    self.quote = quote

    self.data = tt50.loc[(tt50['Weekday'] == self.weekday) & (tt50.index >= self.start_date)]
    self.data['Payment'] = self.payment
    self.data['Purchased units'] = np.floor_divide(self.data['Payment'], self.data[quote]).astype('int64')
    self.data['Cost'] = round(self.data[quote] * self.data['Purchased units']).astype('int64')
    self.data['Commissions'] = round(self.data['Cost'] * 0.001425).astype('int64')
    self.data['Total Cost'] = self.data['Cost'] + self.data['Commissions']

    self.total_payments = len(self.data)
    self.total_cost = self.data['Total Cost'].sum()
    self.fv = round(self.data['Purchased units'].sum() * tt50_price)
    self.sellcommission = round(self.fv * 0.001425)
    self.stt = int(self.fv * 0.003)
    self.net_fv = self.fv - self.sellcommission - self.stt
    self.total_return = self.net_fv / self.total_cost - 1

---

## 設定定期定額條件

In [ ]:
# 每次申購金額
p = 1000

# 自何時開始
s_date = '2021-12-01'

# 按開盤價(Open)、最高價(High)、最低價(Low)、收盤價(Close)計算
q = 'Close'

In [ ]:
mon = Investment(weekday = 1, payment = p, start_date = s_date, quote = q)
tue = Investment(weekday = 2, payment = p, start_date = s_date, quote = q)
wed = Investment(weekday = 3, payment = p, start_date = s_date, quote = q)
thu = Investment(weekday = 4, payment = p, start_date = s_date, quote = q)
fri = Investment(weekday = 5, payment = p, start_date = s_date, quote = q)

In [ ]:
summary = {}
days = [mon, tue, wed, thu, fri]

for i in days:
  summary[i.weekday] = [
              i.total_payments,
              i.total_cost,
              i.net_fv,
              i.total_return
              ]
summary = pd.DataFrame.from_dict(summary, orient='index', 
                  columns = ['付款次數', '成本', '現值', '報酬率'])

max_r = summary['報酬率'].max()
avg_r = summary['報酬率'].mean()
max_r_day = wd_convert[summary.loc[summary['報酬率'] == max_r].index.tolist()[0]]
max_r_alpha = (max_r - avg_r) / avg_r
print(f"自{s_date}起固定在{max_r_day}申購0050，至今擁有最高之報酬率{max_r:.2%}，高於平均達{max_r_alpha:.2%}。")
min_r = summary['報酬率'].min()
min_r_day = wd_convert[summary.loc[summary['報酬率'] == min_r].index.tolist()[0]]
min_r_alpha = (min_r - avg_r) / avg_r
print(f"自{s_date}起固定在{min_r_day}申購0050，至今擁有最低之報酬率{min_r:.2%}，低於平均達{min_r_alpha:.2%}。")

max_fv = summary['現值'].max()
max_fv_day = wd_convert[summary.loc[summary['現值'] == max_fv].index.tolist()[0]]
print(f"自{s_date}起固定在{max_fv_day}申購0050，至今擁有最高之現值{max_fv:,}元。")
min_fv = summary['現值'].min()
min_fv_day = wd_convert[summary.loc[summary['現值'] == min_fv].index.tolist()[0]]
print(f"自{s_date}起固定在{min_fv_day}申購0050，至今擁有最低之現值{min_fv:,}元。")
max_cost = summary['成本'].max()
max_cost_day = wd_convert[summary.loc[summary['成本'] == max_cost].index.tolist()[0]]
min_cost = summary['成本'].min()
min_cost_day = wd_convert[summary.loc[summary['成本'] == min_cost].index.tolist()[0]]
print(f"自{s_date}起固定在{max_cost_day}申購0050，至今投入最多之成本{max_cost:,}元。")
print(f"自{s_date}起固定在{min_cost_day}申購0050，至今投入最少之成本{min_cost:,}元。")

自2021-12-01起固定在星期一申購0050，至今擁有最高之報酬率4.27%，高於平均達12.11%。
自2021-12-01起固定在星期四申購0050，至今擁有最低之報酬率3.53%，低於平均達-7.19%。
自2021-12-01起固定在星期三申購0050，至今擁有最高之現值6,884元。
自2021-12-01起固定在星期一申購0050，至今擁有最低之現值5,837元。
自2021-12-01起固定在星期四申購0050，至今投入最多之成本6,649元。
自2021-12-01起固定在星期一申購0050，至今投入最少之成本5,598元。


In [ ]:
summary

,付款次數,成本,現值,報酬率
1,6,5598,5837,0.042694
2,6,5633,5837,0.036215
3,7,6629,6884,0.038467
4,7,6649,6884,0.035344
5,6,5625,5837,0.037689
